<a href="https://colab.research.google.com/github/SahithReddy-GitHub/Speech-Emotion-Recognition-Using-Deep-Learning/blob/main/SSP_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Model:- MLP Classifier<br>
Dataset :- RAVDESS<br>
Data:- 1440 <br>
Train_Data :- 1080 <br>
Test_Data :- 360<br>
File Type:- .wav <br>
Emotions :- neutral,calm,happy,sad,angry,fearful,disgust,surprise<br>
Accuracy :- 86.7%




In [1]:
!pip install librosa soundfile numpy sklearn pyaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 KB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=bf76f8e34009f5b6a82777da783bd293e926faff1f70d3b5891313e2ff5c22ec
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
Successfully built sklearn
Failed to build pyaudio
ERROR: Could not build wheels for pyaud

In [2]:
!pip install soundfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [4]:
def extract_feature(file_name, mfcc, chroma, mel):
  X, sample_rate= librosa.load(os.path.join(file_name), res_type='kaiser_fast')
  if chroma:
    stft= np.abs(librosa.stft(X))
  result=np.array([])
  if mfcc:
    mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    result=np.hstack((result, mfccs))
  if chroma:
    chroma =np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    result=np.hstack((result, chroma))
  if mel:
    mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    result=np.hstack((result, mel))
  return result

In [5]:
# Emotions in the RAVDESS & TESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
# Emotions to observe
observed_emotions=['neutral','calm','happy','sad','angry','fearful', 'disgust','surprised']

In [6]:
def load_data(test_size=0.2):
  x,y=[],[]
  for file in glob.glob('/content/drive/MyDrive/SSP_PROJECT_DATASET/Actor_23/*.wav'):
    file_name=os.path.basename(file)
    emotion=emotions[file_name.split("-")[2]]
    if emotion not in observed_emotions:
      continue
    feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
    x.append(feature)
    y.append(emotion)
  return train_test_split(np.array(x), y, test_size=test_size, train_size= 0.75,random_state=9)
  /content/drive/MyDrive/SSP_PROJECT_DATASET/audio_speech_actors_01-24
  

<>:12: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<>:12: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<ipython-input-6-1fde5415dd73>:12: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  content/drive/MyDrive/SSP_PROJECT_DATASET/audio_speech_actors_01-24()


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
#split the dataset
import time
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [9]:
#Get the shape of training and testing datasets
print((x_train.shape[0],x_test.shape[0]))


(45, 15)


In [10]:
#Get the number of Features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [11]:
# Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive' , max_iter=500)

In [12]:
#Train the model
model.fit(x_train,y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:611: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(


MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [13]:
#Predict for the test set
y_pred=model.predict(x_test)

In [14]:
# Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#Print the accuracy
print("Accuracy: {:.2f}%" .format(accuracy*100))

Accuracy: 86.67%


In [15]:
##############      CLASSIFICATION REPORT       #####################

from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       angry       1.00      0.75      0.86         4
        calm       1.00      1.00      1.00         2
     fearful       1.00      0.67      0.80         3
       happy       0.50      1.00      0.67         2
         sad       1.00      1.00      1.00         4

    accuracy                           0.87        15
   macro avg       0.90      0.88      0.86        15
weighted avg       0.93      0.87      0.88        15



In [16]:
#CONFUSION MATRIX

from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test,y_pred)
print (matrix)

[[3 0 0 1 0]
 [0 2 0 0 0]
 [0 0 2 1 0]
 [0 0 0 2 0]
 [0 0 0 0 4]]
